### 五个模型的特征并不是太好，因为很多特征重复，都没有三个模型的loss好

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

root_src = 'D:\Dogs vs. Cats'
import os
import shutil
import time
os.chdir(root_src)
import numpy as np
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5","gap_VGG16.h5","gap_VGG19.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

Using TensorFlow backend.


In [2]:
print(X_train.shape,X_test.shape)

(25000, 7168) (12500, 7168)


In [3]:
from keras.models import *
from keras.layers import *

np.random.seed(2017)

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.4)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### 设立停止条件

In [7]:
from keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')

In [9]:
model.fit(X_train, y_train, batch_size=128, nb_epoch=10, validation_split=0.2,callbacks=[earlystopping])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
  640/20000 [..............................] - ETA: 4s - loss: 0.0062 - acc: 0.9984

C:\software\work_tool\Anaconda2\envs\py3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


20000/20000 [==============================] - 4s - loss: 0.0171 - acc: 0.9943 - val_loss: 0.0115 - val_acc: 0.9966
Epoch 2/10
20000/20000 [==============================] - 4s - loss: 0.0150 - acc: 0.9949 - val_loss: 0.0124 - val_acc: 0.9958


In [6]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int((fname.split('\\')[1]).split('.')[0])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('sample_submission_fivemodel.csv', index=None)
df.head(10)

12352/12500 [============================>.] - ETA: 0sFound 12500 images belonging to 1 classes.


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005
